<a href="https://colab.research.google.com/github/karim-mammadov/Computer-Vision-My-projects/blob/main/Celebrity_Face_Image_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download vishesh1412/celebrity-face-image-dataset

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('/content/celebrity-face-image-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
data_path="/content/Celebrity Faces Dataset"

In [ ]:
import tensorflow as tf

In [ ]:
train_data=tf.keras.preprocessing.image_dataset_from_directory(
    data_path,
    validation_split=0.2,
    subset="training",
    seed=13,
    image_size=(224,224),
    batch_size=32,
    label_mode='categorical'
)

valid_data=tf.keras.preprocessing.image_dataset_from_directory(
    data_path,
    validation_split=0.2,
    subset="validation",
    seed=13,
    image_size=(224,224),
    batch_size=32,
    label_mode='categorical'
)

In [ ]:
class_names=train_data.class_names
print(len(class_names))

In [ ]:
for image,label in train_data.take(1):
  for i in range(12):
    print(class_names[tf.argmax(label[i])])

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
for image,label in train_data.take(1):
  for i in range(12):
    plt.subplot(3,4,i+1)
    plt.imshow(image[i].numpy().astype("uint8"))
    plt.title(class_names[tf.argmax(label[i]).numpy()])
    plt.axis("off")

In [ ]:
model=tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(kernel_size=(5,5), filters=32, strides=2, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, strides=2, activation='relu',kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=128, strides=2, activation='relu',kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256, activation='relu',kernel_initializer='he_normal'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=128, activation='relu',kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=len(class_names), activation='softmax')

])

In [ ]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [ ]:
model.fit(train_data, epochs=50, validation_data=valid_data)